In [10]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy.interpolate import interp1d
from scipy.integrate import odeint
from information_scores import *
import os
import scipy.io
import pandas as pd
from scipy.io import loadmat
from pathlib import Path
import math
from scipy.misc import derivative
from scipy import signal

In [11]:
class ReadMat:
    def __init__(self,filename):
        self.input_file = filename
        self.all_data= loadmat(self.input_file)

        self.raw_data= self.all_data['yout']
        self.params= self.all_data['tgParams']

    def positions(self):
        return np.array([self.raw_data[:,10], self.raw_data[:,11], self.raw_data[:,12]])

    def angles(self):
        q0 = np.array(self.raw_data[:,13])
        q1 = np.array(self.raw_data[:,14])
        q2 = np.array(self.raw_data[:,15])
        q3 = np.array(self.raw_data[:,16])
        roll= np.zeros((q0.shape))
        pitch = np.zeros((q0.shape))
        yaw = np.zeros((q0.shape))
        a=0
        for i,j,k,l in zip(q0,q1,q2,q3):

            roll[a]= math.atan2 ( 2*(i*j + k*l), 1-2*(j**2 + k**2))
            val= 2 * (i * k - l * j)
            if abs(val)>=1:
                pitch[a]= math.copysign(math.pi/2,val)
            else:
                pitch[a] = math.asin(val)
            yaw[a] = math.atan2 (2*(i*l + j*k ),1-2*(k**2+l**2))
            a+=1
        return roll, pitch, yaw
    
    def vel(self,xpos,ypos,zpos):
        sampling_rate= xpos.shape[0]
        cut_off_frequency= 240/10*2*np.pi/(sampling_rate/2) #0.054 HZ
        b,a= signal.butter (3, cut_off_frequency) #third order butterworth filter with cutoff frequency given above
        filtered_xpos =signal.filtfilt(b,a,xpos)
        filtered_ypos =signal.filtfilt(b,a,ypos)
        filtered_zpos =signal.filtfilt(b,a,zpos)
        xvel= np.gradient(filtered_xpos)
        yvel= np.gradient(filtered_ypos)
        zvel= np.gradient(filtered_zpos)
        omega1= self.raw_data[:,17]
        omega2= self.raw_data[:,18]
        omega3= self.raw_data[:,19]

        return xvel, yvel, zvel, omega1, omega2, omega3

    def get_states(self):
        roll, pitch, yaw = self.angles()
        xpos, ypos, zpos= self.positions()
        xvel,yvel,zvel, omega1,omega2,omega3 = self.vel(xpos, ypos, zpos)
        return np.array([xpos, ypos, zpos, xvel,yvel,zvel, roll, pitch, yaw, omega1,omega2,omega3])

    def get_actions(self):
        initial_amplitude= (self.params[0][0][5][0][0][2]).item()
        Z_param= self.raw_data[:,6]-initial_amplitude
        roll_param= self.raw_data[:,7]
        pitch_param= self.raw_data[:,8]
        return np.array([Z_param, roll_param, pitch_param])
    
    def sampled_mocap_data(self, samples_steps):
        states_trajectory= self.get_states() # number of state X number of training examples
        action_trajectory= self.get_actions() # number of actions X number of training examples
        new_action_trajectory= np.zeros((action_trajectory.shape[0],int(action_trajectory.shape[1]/samples_steps)))
        new_trajectory= np.zeros((states_trajectory.shape[0],int(states_trajectory.shape[1]/samples_steps)))
        for i in range(int(states_trajectory.shape[1]/samples_steps)):
            new_trajectory[:,i] =np.sum(states_trajectory[:,i*samples_steps:(i+1)*samples_steps],axis=1)/samples_steps
            new_action_trajectory[:,i] =np.sum(action_trajectory[:,i*samples_steps:(i+1)*samples_steps],axis=1)/samples_steps

        return new_trajectory,new_action_trajectory

In [ ]:
# SINDy Function Definitions

def build_library(X):
    
    # obtain size of state space
    n = X.shape[0]
    
    # add in a constant by making a column of ones
    Theta = np.ones((n,1))
    
    # add in first order polynomials of all the states
    for i in range(n):
        state = np.array([X[:,i]]).T
        Theta = np.append(Theta,state,axis=1)
    
    # add in sines of all states
    for i in range(n):
        sine_state = np.array([np.sin(X[:,i])]).T
        Theta = np.append(Theta,sine_state,axis=1)
    
    # add in sines of sum of states 
    
    
 
    return Theta

def Sparse_Regression(Theta,dXdt,thresh):
    
    Xi = np.linalg.lstsq(Theta,dXdt,rcond=None)[0] # Initial guess: Least-squares
    smallinds = np.where(np.abs(Xi) < thresh) # find where the entries is Xi are below the threshold
    Xi[smallinds] = 0 # sparsify!!!!! Hell yeah!!!
    
    return Xi


In [12]:
data_path = '../Raw_data/8_8_2019 to 8_12_2019/'
file1 = '2019-08-08-19-16-02_5sec.mat'
file2 = '2019-08-09-12-49-56_7sec.mat'
file3 = '2019-08-09-13-01-42_10sec.mat'

file_to_open1 = data_path+file1
file_to_open2 = data_path+file2
file_to_open3 = data_path+file3

object1 = ReadMat(file_to_open1)

sampling = 100
# dataset 1
state_traj1, action_traj1 = object1.sampled_mocap_data(sampling)
state_traj1 = state_traj1[:,:500]
action_traj1 = action_traj1[:,:500]

object2 = ReadMat(file_to_open2)
# dataset 2
state_traj2, action_traj2 = object2.sampled_mocap_data(sampling)
state_traj2 = state_traj2[:,:700]
action_traj2 = action_traj2[:,:700]

object3 = ReadMat(file_to_open3)
# dataset 3
state_traj3, action_traj3 = object3.sampled_mocap_data(sampling)
state_traj3 = state_traj3[:,:1000]
action_traj3 = action_traj3[:,:1000]

In [13]:
X = state_traj3

for i in range(X.shape[0])

dX = (1/(12*dt)) * (-X[4:] + 8*X[3:-1] - 8*X[1:-3] + X[:-4])
X1 = X1[2:-2]

dX2 = (1/(12*dt)) * (-X2[4:] + 8*X2[3:-1] - 8*X2[1:-3] + X2[:-4])
X2 = X2[2:-2]

dX = np.vstack([dX1,dX2]) # data matrix

# Trim first and last two that are lost in derivative
X = X[:,2:-2].T
Xprime = dX.T


print(X.shape,Xprime.shape) # print out the shapes of the data matrices as a sanity check



Theta = build_library(X) # build the lbrary of functions (Theta)
threshold = 0.01 # thresholding paramater for sparsification
Xi = Sparse_Regression(Theta,Xprime,threshold) # obtain Xi ("ksi") that gives us the coefficients

print(Xi) # print out Xi as a sanity check

12